In [1]:
import os
directory = r'D:\github\localt20\data\processedData'  # for local
# directory = '/app/dataInHandNow/afterpreprocessed'  # for docker

import polars as pl
matches = pl.read_csv(os.path.join(directory, 'matches.csv'))
matchPlayers = pl.read_csv(os.path.join(directory, 'Matchplayers.csv')).sort('match_id')
playerStats = pl.read_csv(os.path.join(directory, 'playerStats.csv'))
matchPlayers

country,player,player_id,season,match_id
str,str,str,str,i64
"""Australia""","""AC Gilchrist""","""2b6e6dec""","""2005""",211028
"""Australia""","""ML Hayden""","""d8699ab7""","""2005""",211028
"""Australia""","""A Symonds""","""bd77eb62""","""2005""",211028
"""Australia""","""MJ Clarke""","""f842c2cf""","""2005""",211028
"""Australia""","""MEK Hussey""","""48fd7349""","""2005""",211028
…,…,…,…,…
"""Spain""","""EMC Fowler""","""4f88a917""","""2024""",1451294
"""Spain""","""Tashiba Mirza""","""2020d4fd""","""2024""",1451294
"""Spain""","""Memoona Riaz""","""3c5a7deb""","""2024""",1451294


In [2]:
matchPlayers = matchPlayers.with_columns(pl.lit(0).alias('flip'))
matchPlayers

country,player,player_id,season,match_id,flip
str,str,str,str,i64,i32
"""Australia""","""AC Gilchrist""","""2b6e6dec""","""2005""",211028,0
"""Australia""","""ML Hayden""","""d8699ab7""","""2005""",211028,0
"""Australia""","""A Symonds""","""bd77eb62""","""2005""",211028,0
"""Australia""","""MJ Clarke""","""f842c2cf""","""2005""",211028,0
"""Australia""","""MEK Hussey""","""48fd7349""","""2005""",211028,0
…,…,…,…,…,…
"""Spain""","""EMC Fowler""","""4f88a917""","""2024""",1451294,0
"""Spain""","""Tashiba Mirza""","""2020d4fd""","""2024""",1451294,0
"""Spain""","""Memoona Riaz""","""3c5a7deb""","""2024""",1451294,0


In [3]:
# Split each match_id's players into two teams
def swap_teams(match_df):
    team_a = match_df.slice(0, 11)
    team_b = match_df.slice(11, 11)
    return pl.concat([team_b, team_a]).with_columns(pl.lit(1).alias('flip'))

# Group by match_id, apply team swapping, and concatenate with original DataFrame
duplicated_df = (
    matchPlayers.group_by("match_id").map_groups(swap_teams)
)

# Concatenate original and duplicated DataFrames
matchPlayers = pl.concat([matchPlayers, duplicated_df]).sort(['match_id', 'flip']).select(['match_id','flip','player_id','country','player','season'])
pl.Config.set_tbl_rows(44)
matchPlayers.head(44)

match_id,flip,player_id,country,player,season
i64,i32,str,str,str,str
211028,0,"""2b6e6dec""","""Australia""","""AC Gilchrist""","""2005"""
211028,0,"""d8699ab7""","""Australia""","""ML Hayden""","""2005"""
211028,0,"""bd77eb62""","""Australia""","""A Symonds""","""2005"""
211028,0,"""f842c2cf""","""Australia""","""MJ Clarke""","""2005"""
211028,0,"""48fd7349""","""Australia""","""MEK Hussey""","""2005"""
211028,0,"""7d415ea5""","""Australia""","""RT Ponting""","""2005"""
211028,0,"""69762509""","""Australia""","""DR Martyn""","""2005"""
211028,0,"""dd09ff8e""","""Australia""","""B Lee""","""2005"""
211028,0,"""74234d66""","""Australia""","""JN Gillespie""","""2005"""


In [4]:
matchPlayersStats = matchPlayers.join(playerStats, left_on=['player_id','season'] , right_on=['player_id','Season'], how='inner')
pl.Config.set_tbl_rows(10)
matchPlayersStats

match_id,flip,player_id,country,player,season,Player,Mat,Inns,Batting Runs,SR,Ave,Country,Cum batting Ave,Cum battings Runs Total,Cum Inns Total,Cum SR,Overs,Bowling_Runs,Wkts,Econ,Cumulative Overs,Cumulative Wkts,Cumulative Runs,Cumulative Inns,Cumulative Econ,Dis,Ct,St,D/I
i64,i32,str,str,str,str,str,i64,f64,f64,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
211028,0,"""2b6e6dec""","""Australia""","""AC Gilchrist""","""2005""","""AC Gilchrist""",1,1.0,15.0,107.14,15.0,"""Australia""",8.0,16.0,2.0,70.24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
211028,0,"""d8699ab7""","""Australia""","""ML Hayden""","""2005""","""ML Hayden""",1,1.0,6.0,150.0,6.0,"""Australia""",6.0,6.0,1.0,150.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
211028,0,"""bd77eb62""","""Australia""","""A Symonds""","""2005""","""A Symonds""",1,1.0,0.0,0.0,0.0,"""Australia""",16.0,32.0,2.0,123.08,3.0,14.0,2.0,4.66,6.0,3.0,47.0,2.0,23.5,1.0,1.0,0.0,0.5
211028,0,"""f842c2cf""","""Australia""","""MJ Clarke""","""2005""","""MJ Clarke""",1,1.0,0.0,0.0,0.0,"""Australia""",3.5,7.0,2.0,87.5,3.0,25.0,1.0,8.33,5.0,1.0,36.0,2.0,18.0,0.0,0.0,0.0,0.0
211028,0,"""48fd7349""","""Australia""","""MEK Hussey""","""2005""","""MEK Hussey""",1,1.0,1.0,16.66,1.0,"""Australia""",16.0,32.0,2.0,111.66,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0,0.0,1.5
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
1450767,1,"""f22d2247""","""Japan""","""KH Abe""","""2024""","""KH Abe""",12,10.0,206.0,163.49,25.75,"""Japan""",17.03,255.0,18.0,115.31,0.0,0.0,0.0,0.0,1.0,1.0,8.0,1.0,8.0,3.0,3.0,0.0,0.142857
1450767,1,"""fef40097""","""Japan""","""W Miyauchi""","""2024""","""W Miyauchi""",12,8.0,166.0,110.66,27.66,"""Japan""",19.69,298.0,19.0,93.55,1.0,3.0,0.0,3.0,1.0,0.0,3.0,1.0,3.0,18.0,12.0,6.0,0.782609
1450767,1,"""96275259""","""Japan""","""B Ito-Davis""","""2024""","""B Ito-Davis""",10,7.0,132.0,138.94,26.4,"""Japan""",26.4,132.0,7.0,138.94,20.2,69.0,14.0,3.39,20.2,14.0,69.0,9.0,7.666667,6.0,6.0,0.0,0.6


In [5]:
match_id = matchPlayersStats['match_id','player_id'].with_columns(pl.lit(1).alias("one")).group_by('match_id').agg(pl.sum('one')).filter(pl.col('one') == 44)['match_id']
# matchPlayersStats['match_id','player_id'].with_columns(pl.lit(1).alias("one")).group_by('match_id').agg(pl.sum('one'))['one'].value_counts().sort(by='one')
match_id

match_id
i64
211028
211048
225263
225271
226374
…
1447492
1447496
1450753


In [6]:
matchPlayersStats = matchPlayersStats.filter(pl.col('match_id').is_in(match_id))
matchPlayersStats

match_id,flip,player_id,country,player,season,Player,Mat,Inns,Batting Runs,SR,Ave,Country,Cum batting Ave,Cum battings Runs Total,Cum Inns Total,Cum SR,Overs,Bowling_Runs,Wkts,Econ,Cumulative Overs,Cumulative Wkts,Cumulative Runs,Cumulative Inns,Cumulative Econ,Dis,Ct,St,D/I
i64,i32,str,str,str,str,str,i64,f64,f64,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
211028,0,"""2b6e6dec""","""Australia""","""AC Gilchrist""","""2005""","""AC Gilchrist""",1,1.0,15.0,107.14,15.0,"""Australia""",8.0,16.0,2.0,70.24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
211028,0,"""d8699ab7""","""Australia""","""ML Hayden""","""2005""","""ML Hayden""",1,1.0,6.0,150.0,6.0,"""Australia""",6.0,6.0,1.0,150.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
211028,0,"""bd77eb62""","""Australia""","""A Symonds""","""2005""","""A Symonds""",1,1.0,0.0,0.0,0.0,"""Australia""",16.0,32.0,2.0,123.08,3.0,14.0,2.0,4.66,6.0,3.0,47.0,2.0,23.5,1.0,1.0,0.0,0.5
211028,0,"""f842c2cf""","""Australia""","""MJ Clarke""","""2005""","""MJ Clarke""",1,1.0,0.0,0.0,0.0,"""Australia""",3.5,7.0,2.0,87.5,3.0,25.0,1.0,8.33,5.0,1.0,36.0,2.0,18.0,0.0,0.0,0.0,0.0
211028,0,"""48fd7349""","""Australia""","""MEK Hussey""","""2005""","""MEK Hussey""",1,1.0,1.0,16.66,1.0,"""Australia""",16.0,32.0,2.0,111.66,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0,0.0,1.5
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
1450765,1,"""073126d6""","""Tanzania""","""LS Barkania""","""2024""","""LS Barkania""",5,1.0,9.0,56.25,9.0,"""Tanzania""",9.0,9.0,1.0,56.25,13.5,42.0,10.0,3.03,13.5,10.0,42.0,5.0,8.4,1.0,1.0,0.0,0.2
1450765,1,"""c06c25fd""","""Tanzania""","""Akhil Anil""","""2024""","""Akhil Anil""",5,3.0,49.0,125.64,24.5,"""Tanzania""",17.42,265.0,23.0,119.06,7.0,25.0,4.0,3.57,63.8,23.0,409.0,22.0,18.590909,6.0,6.0,0.0,0.230769
1450765,1,"""0848ac5f""","""Tanzania""","""S Thakor""","""2024""","""S Thakor""",5,2.0,9.0,112.5,4.5,"""Tanzania""",14.14,477.0,44.0,138.44,15.0,52.0,10.0,3.46,177.0,65.0,980.0,56.0,17.5,19.0,19.0,0.0,0.339286


In [7]:
matchPlayersStats = matchPlayersStats.drop(['country','player','player_id','season','Player','Country'])
matchPlayersStats

match_id,flip,Mat,Inns,Batting Runs,SR,Ave,Cum batting Ave,Cum battings Runs Total,Cum Inns Total,Cum SR,Overs,Bowling_Runs,Wkts,Econ,Cumulative Overs,Cumulative Wkts,Cumulative Runs,Cumulative Inns,Cumulative Econ,Dis,Ct,St,D/I
i64,i32,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
211028,0,1,1.0,15.0,107.14,15.0,8.0,16.0,2.0,70.24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
211028,0,1,1.0,6.0,150.0,6.0,6.0,6.0,1.0,150.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
211028,0,1,1.0,0.0,0.0,0.0,16.0,32.0,2.0,123.08,3.0,14.0,2.0,4.66,6.0,3.0,47.0,2.0,23.5,1.0,1.0,0.0,0.5
211028,0,1,1.0,0.0,0.0,0.0,3.5,7.0,2.0,87.5,3.0,25.0,1.0,8.33,5.0,1.0,36.0,2.0,18.0,0.0,0.0,0.0,0.0
211028,0,1,1.0,1.0,16.66,1.0,16.0,32.0,2.0,111.66,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0,0.0,1.5
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
1450765,1,5,1.0,9.0,56.25,9.0,9.0,9.0,1.0,56.25,13.5,42.0,10.0,3.03,13.5,10.0,42.0,5.0,8.4,1.0,1.0,0.0,0.2
1450765,1,5,3.0,49.0,125.64,24.5,17.42,265.0,23.0,119.06,7.0,25.0,4.0,3.57,63.8,23.0,409.0,22.0,18.590909,6.0,6.0,0.0,0.230769
1450765,1,5,2.0,9.0,112.5,4.5,14.14,477.0,44.0,138.44,15.0,52.0,10.0,3.46,177.0,65.0,980.0,56.0,17.5,19.0,19.0,0.0,0.339286


In [8]:
matchPlayersStats.shape

(52800, 24)

In [ ]:
import numpy as np
directory = r'D:\github\localt20\data\aftermerging'  # for local
matchPlayersStats.write_csv(os.path.join(directory, 'playersStatsflip.csv'))